In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
links_page_result = requests.get('https://www.your.md/your-symptoms')
print(links_page_result.status_code)
print(links_page_result.headers)

200
{'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Fri, 21 Jun 2019 02:24:07 GMT', 'Server': 'nginx', 'Last-Modified': 'Wed, 19 Jun 2019 14:17:24 GMT', 'ETag': 'W/"3cb5c6dfe75befe1377a8b6dae4096ac"', 'Cache-Control': 'public, max-age=600, proxy-revalidate', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Age': '257', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 14484a063800eaed878a3068abf4dfac.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'FRA54', 'X-Amz-Cf-Id': 'sqxuFxu7Eyhk2CjlxqnhqexkAH9hxko3OqzumakWxWaPFKItzRsI6w=='}


In [3]:
links_page_content = links_page_result.content
links_page_soup = BeautifulSoup(links_page_content, 'html.parser')

In [4]:
ul_of_links = links_page_soup.find("section", {"class": "list"})

In [5]:
list_of_a_tags = ul_of_links.find_all('a')

In [6]:
list_of_href = [link.attrs['href'] for link in list_of_a_tags]

In [7]:
list_of_href = ['https://www.your.md' + link for link in list_of_href]

In [8]:
list_of_alphabets = [link.text for link in list_of_a_tags]
print(list_of_alphabets[0:5])

['Abdominal Pain Spreading To Shoulder Blade, Right Sided Abdominal Pain, Upper Abdominal Pain, Vomiting, Sweating, Feeling Sick Nausea and Loss Of Appetite', 'Abdominal Pain Worse On Pressing, Right Sided Abdominal Pain, Vomiting, Feeling Sick Nausea, High Temperature and Loss Of Appetite', 'Abdominal pain', 'Abdominal pain after eating', 'Abdominal pain and Bloody diarrhoea']


In [9]:
links_data = pd.DataFrame({'symptom' : list_of_alphabets,'url' : list_of_href,})
print(links_data.head())

                                             symptom                                                url
0  Abdominal Pain Spreading To Shoulder Blade, Ri...  https://www.your.md/your-symptoms/pain-radiati...
1  Abdominal Pain Worse On Pressing, Right Sided ...  https://www.your.md/your-symptoms/abdominal-pa...
2                                     Abdominal pain  https://www.your.md/your-symptoms/abdominal-pain/
3                        Abdominal pain after eating  https://www.your.md/your-symptoms/postprandial...
4                Abdominal pain and Bloody diarrhoea  https://www.your.md/your-symptoms/abdominal-pa...


In [10]:
links_data.shape

(1035, 2)

In [11]:
symptoms = []
disease_name = []
count = 0
for each_disease_link in list_of_href:
    
    each_disease_req = requests.get(each_disease_link)
    disease_soup = BeautifulSoup(each_disease_req.content, 'html.parser')
    
    heading = disease_soup.find('h1').text
    
    sections = disease_soup.find("section", {"class": "bigfoot-section"})
    try:
        name = sections.find('ul').text.strip()
    except:
        ValueError
    name = name.replace("  ", "")
    name = name.replace("\n\n\n\n\n", ",")
    #name = name.to_list()
    
    #name = name.split(',')
    symptoms.append(heading)
    disease_name.append(name)
    
    
    
    count=count+1

In [12]:
links_data = pd.DataFrame({'symptom' : symptoms,'disease' : disease_name})
#links_data = links_data.reset_index()
links_data.head()

,symptom,disease
0,"Abdominal Pain Spreading To Shoulder Blade, Ri...","Acute Cholecystitis,Gallstone"
1,"Abdominal Pain Worse On Pressing, Right Sided ...","Acute Cholecystitis,Appendicitis"
2,Abdominal pain,"Bowel Cancer,Constipation,Crohn's disease,Glut..."
3,Abdominal pain after eating,"Acid reflux,Gallstone,Irritable bowel syndrome..."
4,Abdominal pain and Bloody diarrhoea,"Crohn's disease,Ulcerative colitis"


In [13]:
links_data = links_data[links_data["symptom"].str.contains('and')==False].reset_index()

In [14]:
links_data.drop(['index'],axis=1,inplace=True)

In [15]:
final_new=links_data.disease.str.split(pat = ",",expand=True)

In [16]:
new_one = final_new.merge(links_data, left_index = True, right_index = True)
new_one = new_one.drop(["disease"],axis=1)
new_one = new_one.melt(id_vars = ["symptom"], value_name = "disease")

In [17]:
new_one = new_one.sort_values(by='symptom')
new_one = new_one.dropna()
new_one = new_one.drop(['variable'],axis=1)
new_one = new_one.reset_index().drop(['index'],axis=1)

In [18]:
new_one.to_csv('new.csv',index=False)

In [19]:
symptoms = []
disease_name = []
count = 0
for each_disease_link in list_of_href:
    
    each_disease_req = requests.get(each_disease_link)
    disease_soup = BeautifulSoup(each_disease_req.content, 'html.parser')
    
    heading = disease_soup.find('h1').text
    sections = disease_soup.find("section", {"class": "bigfoot-section"})
    try:
        name = sections.find('ul').text.strip()
    except:
        ValueError
    name = name.replace("  ", "")
    name = name.replace("\n\n\n\n\n", ",")
    name = name.split(',')
    p = len(name)
    
    sections = disease_soup.find_all("section", {"class": "bigfoot-section"})[1:p]
    
    
    
    count=count+1
    if(count==3):
        break

In [20]:
sections

[<section class="bigfoot-section">
 <h2 id="cancer-of-the-colon-rectum-or-bowel">Bowel Cancer</h2>
 <p>
                                     Bowel cancer is a general term for cancer that begins in the large bowel.
                                 </p>
 <br/>
 <!-- <p>
                             2. Inteligent paragraph generated with full Condition data
                             </p>
                             <br> -->
 <h4>Symptoms of this condition may include:</h4>
 <ul>
 <li>Diarrhoea</li>
 <li>Blood in stool or poo</li>
 <li>Abdominal pain</li>
 <li>Change in bowel habit</li>
 <li>Bloody diarrhoea</li>
 </ul>
 <p>Our Health A-Z contains more information on <a href="/condition/cancer-of-the-colon-rectum-or-bowel/#symptoms"><b>Bowel Cancer</b></a>.</p>
 <p><a href="https://webapp.your.md/login?utm_source=yourmd&amp;utm_medium=in-article&amp;utm_campaign=your-symptoms" target="_blank">If you are worried about this condition you can try our free AI Symptom Checker here.</a></p>